In [1]:
from langchain_community.llms import Ollama
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser

In [2]:
llm=Ollama(model="llama3")
loader=PyPDFLoader(file_path=r"C:\Users\Garv Khurana\OneDrive\Desktop\2023_CURRENT_Medical_Diagnosis_and_Treat - Copy_removed.pdf")
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)

In [3]:
data=loader.load()
data[:5]

[Document(metadata={'source': 'C:\\Users\\Garv Khurana\\OneDrive\\Desktop\\2023_CURRENT_Medical_Diagnosis_and_Treat - Copy_removed.pdf', 'page': 0}, page_content='Copyright © 2023 by McGraw Hill. All rights reserved. Except as permitted under the United States Copyright Act of 1976,\nno part of this publication may be reproduced or distributed in any form or by any means, or stored in a database or retrieval\nsystem, without the prior written permission of the publisher.\nISBN: 978-1-26-468774-9\nMHID: 1-26-468774-5\nThe material in this eBook also appears in the print version of this title: ISBN: 978-1-26-468734-3,\nMHID: 1-26-468734-6.\neBook conversion by codeMantra\nVersion 1.0\nAll trademarks are trademarks of their respective owners. Rather than put a trademark symbol after every occurrence of a\ntrademarked name, we use names in an editorial fashion only, and to the bene\ue018t of the trademark owner, with no intention of\ninfringement of the trademark. Where such designations a

In [4]:
content=splitter.split_documents(data)

In [5]:
content[:2]

[Document(metadata={'source': 'C:\\Users\\Garv Khurana\\OneDrive\\Desktop\\2023_CURRENT_Medical_Diagnosis_and_Treat - Copy_removed.pdf', 'page': 0}, page_content='Copyright © 2023 by McGraw Hill. All rights reserved. Except as permitted under the United States Copyright Act of 1976,\nno part of this publication may be reproduced or distributed in any form or by any means, or stored in a database or retrieval\nsystem, without the prior written permission of the publisher.\nISBN: 978-1-26-468774-9\nMHID: 1-26-468774-5\nThe material in this eBook also appears in the print version of this title: ISBN: 978-1-26-468734-3,\nMHID: 1-26-468734-6.\neBook conversion by codeMantra\nVersion 1.0\nAll trademarks are trademarks of their respective owners. Rather than put a trademark symbol after every occurrence of a\ntrademarked name, we use names in an editorial fashion only, and to the bene\ue018t of the trademark owner, with no intention of\ninfringement of the trademark. Where such designations a

In [10]:
db=FAISS.from_documents(content,OllamaEmbeddings(model="llama3"))

In [11]:
retriever=db.as_retriever()

In [15]:
retriever=db.as_retriever()

In [19]:
db.similarity_search('we use names in an editorial fashion only')

[Document(metadata={'source': 'C:\\Users\\Garv Khurana\\OneDrive\\Desktop\\2023_CURRENT_Medical_Diagnosis_and_Treat - Copy_removed.pdf', 'page': 12}, page_content='some instances, full-text reference articlesE-\nCHAPTERS, CMDT ONLINE , & AVAILABLE APPSSeven\n e-chapters listed in the Table of Contents can be accessed at www.AccessMedicine.com/CMDT. These online-only\nchapters (available without need for subscription) include\n• Anti-Infective Chemotherapeutic & Antibiotic Agents\n• Diagnostic Testing & Medical Decision Making\n• Information Technology in Patient Care\n• Integrative Medicine\n• Podiatric Disorders\n• Women’s Health Issues\n• Appendix: Therapeutic Drug Monitoring, Laboratory Reference Intervals, & Commonly Used Blood Sp ecimen\nCollection Tubes\nInstitutional or individual subscriptions to AccessMedicine also have full electronic access to CMDT 2023 . Subscribers to\nCMDT Online receive full electronic access to CMDT 2023 as well as\n• An expanded, dedicated media galler

In [20]:
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

In [21]:
prompt=ChatPromptTemplate.from_template("""
you are a doctor and a researcher as well and specialized in doing medical tasks answer the following questions 
based on the context provided,think step by step before answering 
I will tip you $1000 if I find the answer helpful

<context>
{context}
</context>

Question:{input}
"""
    
)

In [23]:
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nyou are a doctor and a researcher as well and specialized in doing medical tasks answer the following questions \nbased on the context provided,think step by step before answering \nI will tip you $1000 if I find the answer helpful\n\n<context>\n{context}\n</context>\n\nQuestion:{input}\n'))])
| Ollama(model='llama3')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [25]:
chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
chain.invoke({'input':'Differential diagnosos'})